In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn import preprocessing
from google.colab import files
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import VotingRegressor

In [ ]:
oportunidades = pd.read_csv('/content/short_df_Encoded.csv')
oportunidades.drop(columns= {'Unnamed: 0'}, inplace=True)
test = pd.read_csv('/content/DataFrame_test_encoded.csv')
oportunidades.head()

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,0.0,0.0,1.0,0.0,0.0,15.635076,10.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,10.940740,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,11.336971,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,15.819943,14.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,16.407566,25.0,0.0,1.0,0.0


In [ ]:
test.head()

,Unnamed: 0,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Opportunity_ID,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,0,0.0,0.0,1.0,0.0,0.0,10689.0,11.905436,0.0,1.0,1.0
1,1,0.0,0.0,1.0,0.0,0.0,10690.0,11.787392,0.0,1.0,1.0
2,2,0.0,1.0,0.0,0.0,0.0,10691.0,9.954062,0.0,1.0,1.0
3,3,0.0,1.0,0.0,0.0,0.0,10692.0,12.338534,1.0,1.0,1.0
4,4,0.0,1.0,0.0,0.0,0.0,10693.0,8.657390,0.0,0.0,0.0


In [ ]:
oportunidades = oportunidades[{'Region_APAC','Region_Americas','Region_EMEA','Region_Japan','Region_Middle East','Total_Amount','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr','Decision'}]

In [ ]:
X = oportunidades.loc[:,oportunidades.columns != 'Decision'].values
y = oportunidades.loc[:,'Decision'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n_neighbors = 35

knn = KNeighborsRegressor(n_neighbors, metric='minkowski', weights='uniform', algorithm='auto')
knn.fit(X_train, y_train)
print('Accuracy of K-NN  on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN  on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

Accuracy of K-NN  on training set: 0.62
Accuracy of K-NN  on test set: 0.58


In [ ]:
pred = knn.predict(X_test)
#print(confusion_matrix(y_test, pred))
#print(classification_report(y_test, pred))

In [ ]:
loss = log_loss(y_test, pred)
print(f"Log Loss is {loss}")


Log Loss is 0.3482621240920133


In [ ]:
subir = pd.DataFrame()
subir['target'] = pred

subir.describe()

,target
count,1172.000000
mean,0.498464
std,0.383329
min,0.000000
25%,0.142857
50%,0.457143
75%,0.914286
max,1.000000


In [ ]:
test.head()

,Unnamed: 0,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Opportunity_ID,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,0,0.0,0.0,1.0,0.0,0.0,10689.0,11.905436,0.0,1.0,1.0
1,1,0.0,0.0,1.0,0.0,0.0,10690.0,11.787392,0.0,1.0,1.0
2,2,0.0,1.0,0.0,0.0,0.0,10691.0,9.954062,0.0,1.0,1.0
3,3,0.0,1.0,0.0,0.0,0.0,10692.0,12.338534,1.0,1.0,1.0
4,4,0.0,1.0,0.0,0.0,0.0,10693.0,8.657390,0.0,0.0,0.0


In [ ]:
test = test[{'Unnamed: 0','Region_APAC','Region_Americas','Region_EMEA','Region_Japan','Region_Middle East','Opportunity_ID','Total_Amount','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr'}]

In [ ]:
test.dtypes

Unnamed: 0                              int64
Region_Japan                          float64
Opportunity_ID                        float64
Region_Americas                       float64
Region_APAC                           float64
Pricing, Delivery_Terms_Approved      float64
Region_EMEA                           float64
Region_Middle East                    float64
Pricing, Delivery_Terms_Quote_Appr    float64
Total_Amount                          float64
dtype: object

In [ ]:
test_limpio =  test.drop(columns= {'Unnamed: 0'})
test_limpio =  test_limpio.drop(columns= {'Opportunity_ID'})

In [ ]:
test_limpio.head()

,Region_Japan,Region_Americas,Region_APAC,"Pricing, Delivery_Terms_Approved",Region_EMEA,Region_Middle East,"Pricing, Delivery_Terms_Quote_Appr",Total_Amount
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,11.905436
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,11.787392
2,0.0,1.0,0.0,1.0,0.0,0.0,1.0,9.954062
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,12.338534
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8.657390


In [ ]:
X = preprocessing.scale(test_limpio)

In [ ]:
pred = knn.predict(X)
pred

array([0.91428571, 0.91428571, 0.68571429, ..., 0.68571429, 0.68571429,
       0.14285714])

In [ ]:
subir = pd.DataFrame()
subir['Opportunity_ID'] = test['Opportunity_ID']
subir['target'] = pred
subir['Opportunity_ID'] = pd.to_numeric(subir['Opportunity_ID'], downcast='integer')
subir.sort_values('Opportunity_ID',inplace = True)
subir = subir.drop_duplicates('Opportunity_ID',keep = 'last')
subir.set_index('Opportunity_ID', inplace = True)
subir

,target
Opportunity_ID,
10689,0.914286
10690,0.914286
10691,0.685714
10692,0.628571
10693,0.914286
...,...
12364,0.914286
12365,0.685714
12366,0.685714


In [ ]:
subir.value_counts()

target  
0.914286    436
0.685714    276
1.000000    167
0.971429    165
0.085714     94
0.485714     61
0.142857     59
0.171429     47
0.428571     32
0.000000     29
0.600000     28
0.828571     21
0.228571     16
0.200000     14
0.314286     11
0.857143     10
0.514286      9
0.942857      9
0.457143      9
0.285714      9
0.028571      7
0.657143      6
0.114286      6
0.885714      6
0.771429      5
0.800000      5
0.714286      4
0.257143      4
0.400000      4
0.542857      4
0.571429      4
0.342857      4
0.742857      3
0.628571      3
dtype: int64

In [ ]:
subir.describe()

,target
count,1567.000000
mean,0.694940
std,0.312918
min,0.000000
25%,0.485714
50%,0.857143
75%,0.914286
max,1.000000


In [ ]:
csv3 = subir.to_csv('knn_pred_1.csv')
#files.download('knn_pred_1.csv')